In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '../..')

#### Load mesh and key points

In [ ]:
# load kps
from UltraMotionCapture import kps, obj3d

vicon = kps.MarkerSet()
vicon.load_from_vicon('../../UltraMotionCapture/data/6kmh_softbra_8markers_1.csv')
vicon.interp_field()

points_vicon = vicon.get_frame_coord(0)
pvpcd_vicon = obj3d.np2pvpcd(points_vicon, radius=10, phi_resolution=10, theta_resolution=10)

In [ ]:
# load parameters
import numpy as np
from UltraMotionCapture import field

r = np.load('../../UltraMotionCapture/config/calibrate/r.npy')
s = np.load('../../UltraMotionCapture/config/calibrate/s.npy')
t = np.load('../../UltraMotionCapture/config/calibrate/t.npy')
s, m = field.transform_rst2sm(r, s, t)

In [ ]:
# load mesh
import pyvista as pv
pv.set_jupyter_backend('pythreejs')

mesh = pv.read('../../UltraMotionCapture/data/6kmh_softbra_8markers_1/speed_6km_soft_bra.000001.obj')
texture = pv.read_texture('../../UltraMotionCapture/data/6kmh_softbra_8markers_1/speed_6km_soft_bra.000001.jpg')

mesh.transform(m)
mesh.scale(s)

In [ ]:
# plot mesh and kps
scene = pv.Plotter()
scene.add_mesh(mesh, show_edges=True)
scene.add_mesh(pvpcd_vicon, color='Blue')
scene.show()

#### Fix Dis-connectivity Problem

In [ ]:
# mesh.triangulate()
clean = mesh.clean()
bodies = clean.split_bodies(label=True)
bodies.plot(multi_colors=True, opacity=0.25)

In [ ]:
name = bodies.keys()[0]
points = bodies[name]
surf = points.extract_surface()
surf.plot()

#### Calculate Geodesic Paths

In [ ]:
kps_id =[surf.find_closest_point(tuple(kp)) for kp in points_vicon]

In [ ]:
geod_paths = {}
geod_dists = {}

for i in range(len(kps_id)):
    for j in range(i):
        name = '{} to {}'.format(i, j)
        geod_paths[name] = surf.geodesic(kps_id[i], kps_id[j])
        geod_dists[name] = surf.geodesic_distance(kps_id[i], kps_id[j])

In [ ]:
# print geodesic distances
geod_dists

In [ ]:
geod_paths_ls = list(geod_paths.values())
geods = sum(geod_paths_ls[1:], start=geod_paths_ls[0])

scene = pv.Plotter()
scene.add_mesh(mesh)
scene.add_mesh(geods, line_width=1000, color='blue')
scene.show()